In [35]:
import numpy as np




M = np.array([[0.5, 0.2, 0.2, 0.2,  0.2],
              [0.2, 0.3, 0.5, 0.4,  0.2], 
              [0.2, 0.7, 0.5, 0.2,  0.2], 
              [0.3, 0.2, 0.2, 0.6,  0.2],
              [0.2, 0.2, 0.2, 0.2,  1.0]], dtype=float)

s = np.array([1.0, 1.0, 1.0, 1.0, 1.0])

for _ in range(10):
    s = np.matmul(M, s)
    print (sum(s))
    print (s)






7.999999999999999
[1.3 1.6 1.8 1.5 1.8]
12.899999999999999
[1.99 2.6  2.94 2.33 3.04]
20.85
[3.177 4.188 4.762 3.711 5.012]
33.727000000000004
[5.1231 6.7596 7.6926 5.9721 8.1796]
54.5745
[ 8.28233 10.92356 12.43298  9.64655 13.28908]
88.32055000000001
[13.399599 17.66646  20.106574 15.601753 21.546164]
142.94219300000003
[21.6839897 28.5830788 32.5293122 25.2447711 34.9010412]
231.35091910000006
[35.09363551 46.25449436 52.63877166 40.85474601 56.50927156]
374.4445977700001
[56.79827447 74.85821396 85.1890625  66.12144578 91.47760107]
606.0472217510003
[ 91.9284019  121.15574885 137.87474528 107.01732531 148.07100041]


## Numpy Version

The graph is fully connected with edge weight 1/N. Each column sums to 1. In order to unbalance the graph in favor of your neighbor a node subtracts from his inloop edge and attributes this offset to his connection. For example:

n_i --> n_i 0.9
n_i --> n_j 0.1

Mii = 1/N - (1/N) * (1 - 0.9)
Mij = 1/N + (1/N) * 0.1




In [3]:
import numpy as np

s = np.array([1.0, 10.0, 1.0, 1.0, 1.0])
ns = s / np.linalg.norm(s, 1)
print (ns)



[0.07142857 0.71428571 0.07142857 0.07142857 0.07142857]


In [58]:

def boltrank(M, s):    
    v = s
    ns = s / np.linalg.norm(s, 1)
    for _ in range(100):
        v = np.matmul(M, v)
        # v = v / np.linalg.norm(v, 1) 
    v = np.multiply(v, s)
    v = v / np.linalg.norm(v, 1)
    attr = v
    e = np.sum(s) * 0.1
    print (e)
    v = np.multiply(v, e)
    v = s + v
    return v, attr



# Edge matrix.
N = 5.0
ea = 1/N - (1/N) * (1 - 0.0)
eb = 1/N + (1/N) * 1.0

M = np.array([[1/N,  0.71428571/N, 0.07142857/N, 0.07142857/N,  1/N],
              [0.07142857/N, ea,   0.07142857/N, 0.07142857/N,  1/N], 
              [0.07142857/N, 0.71428571*eb,  1/N, 0.07142857/N,  1/N], 
              [0.07142857/N, 0.71428571/N, 0.07142857/N,  ea,  1/N],
              [0.07142857/N, 0.71428571/N, 0.07142857/N,  eb,  1/N]], dtype=float)


s = np.array([1.0, 2.0, 1.0, 1.0, 1.0])

# Run 100 steps.
steps = 1
s_init = s
print ('Initial stake:', s_init)
for i in range(steps):
    s, attr = boltrank(M, s)
    
    print ('Attr at ' + str(i+1) + ":", attr)
    print ('Stake at ' + str(i+1) + ":", s)
    
print ('Raw attributions: ', attr)
total_increase = s - s_init
print ('Total increase: ', total_increase)
value_increase = np.divide(s, s_init)
print ('% Increase in stake: ', value_increase/100)
norm_value_increase = value_increase / np.linalg.norm(value_increase, 1)
print ('Normalized value increase:', norm_value_increase)

print ((norm_value_increase[1] - norm_value_increase[2]) / (norm_value_increase[3] - norm_value_increase[4]))



Initial stake: [1. 2. 1. 1. 1.]
0.6000000000000001
Attr at 1: [0.15493875 0.26296582 0.27221794 0.13148291 0.17839459]
Stake at 1: [1.09296325 2.15777949 1.16333077 1.07888974 1.10703675]
Raw attributions:  [0.15493875 0.26296582 0.27221794 0.13148291 0.17839459]
Total increase:  [0.09296325 0.15777949 0.16333077 0.07888974 0.10703675]
% Increase in stake:  [0.01092963 0.0107889  0.01163331 0.0107889  0.01107037]
Normalized value increase: [0.19796077 0.19541174 0.21070595 0.19541174 0.20050981]
2.9999999999999836


In [123]:
import copy 

def normalize(M):
    result = {}
    sum_m = sum(M.values())
    for id in M.keys():
        if sum_m == 0:
             result[id] = 1 / len(M)
        else:
             result[id] = (M[id] / sum_m)
    return result

def multiply(A, B):
    result = {}
    for id in A.keys():
        result[id] = A[id] * B[id]
    return result       


class Bittensor:
    
    def __init__(self):
        self.S = {} # id --> stake
        self.W = {} # id --> weights (id, w)
        self.A = {} # id --> attr
        self.n = len(self.S.values())
        self.supply = sum(self.S.values())
        
    def init(self, id):
        self.S[id] = 0
        self.W[id] = None

    def set_weights(self, id, ids, weights):
        assert (len(weights) != 0)
        assert (len(ids) == len(weights))
        assert (abs(sum(weights) - 1.0) < 0.001)        
        self.W[id] = list(zip(ids, weights))
        
        
    def matmul(W, b):
        result = {}

        dem = (1 / len(W))

        for id in W.keys():
            result[id] = sum(b.values()) * dem

        for id in W.keys():
            if not W[id]:
                continue
            for w in W[id]:
                if w[0] == id:
                    result[id] -= (b[id] * (1 - w[1])) * dem
                else:            
                    result[w[0]] += (b[id] * w[1]) * dem

        return result

        
    def emit(self, steps=100):
        
        v = normalize(self.S)
        for _ in range(steps):
            v = matmul(self.W, v) 
        v = multiply(v, self.S)
        v = normalize(v)
        
        emmission = self.supply * 0.01 + 0.01
        for id in v:
            self.S[id] += emmission * v[id]

    def __str__(self):
        strng = "" 
        strng += "Stake: " + str([id + ": %.3f" % self.S[id] for id in self.S.keys()]) + "\n"
        strng += "Weights{ \n"
        for id in self.S.keys():
            strng += "\t"
            strng += str(id)
            strng += ": "
            if self.W[id] == None:
                strng += "nil"
                strng += " "
            else:
                for w in self.W[id]:
                    wstr = str(w[0]) + " " + str(w[1])
                    strng += wstr
                    strng += " "

            strng += "\n"
        strng += '}'
        return strng




SyntaxError: invalid syntax (<ipython-input-123-1fbcff800814>, line 1)

In [124]:
b = Bittensor()
b.init('a')
b.init('b')
b.init('c')
b.init('d')
b.init('e')


b.set_weights('a', ['a', 'b'], [0.9, 0.1])
b.set_weights('d', ['d', 'c'], [0.80, 0.2])
b.set_weights('c', ['c', 'b', 'a'], [0.5, 0.25, 0.25])
b.set_weights('e', ['e', 'd', 'c'], [0.6, 0.3, 0.1])

print (b)

for _ in range(1000):
    b.emit()
print (b)


Stake: ['a: 0.000', 'b: 0.000', 'c: 0.000', 'd: 0.000', 'e: 0.000']
Weights{ 
	a: a 0.9 b 0.1 
	b: nil 
	c: c 0.5 b 0.25 a 0.25 
	d: d 0.8 c 0.2 
	e: e 0.6 d 0.3 c 0.1 
}
Stake: ['a: 2.272', 'b: 2.957', 'c: 1.497', 'd: 2.095', 'e: 1.179']
Weights{ 
	a: a 0.9 b 0.1 
	b: nil 
	c: c 0.5 b 0.25 a 0.25 
	d: d 0.8 c 0.2 
	e: e 0.6 d 0.3 c 0.1 
}
